In [1]:
from dotenv import load_dotenv

from frames.data.figstep import QueryType, SafeBenchLanguages
from frames.utils.settings import load_models
from frames.experiments import MultilingualModelGenerator

load_dotenv()

MODELS = load_models().to_dict(orient="index").values()
QUERY_TYPES = [QueryType.baseline]
GUIDE = ["safe.n.01", "harm.v.01"]
DB_PATH = "results.shelf"

generator = MultilingualModelGenerator(
    db_path=DB_PATH,
    models=MODELS,
    query_types=QUERY_TYPES,
    languages=SafeBenchLanguages,
    guide=GUIDE,
)
generator.generate_all()

RuntimeError: Failed to load model: hugging-quants/Meta-Llama-3.1-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`